In [68]:
import pandas as pd

In [69]:
df1=pd.read_csv('orders.csv')
df2=pd.read_csv('warehouse.csv')

In [70]:
df1.head()

,order_id,customer_id,warehouse_id,order_date,shipper_date
0,789,3731,8118,01/01/2019,01/04/2019
1,790,3486,8118,1/1/2019,1/4/2019
2,791,2623,8118,1/1/2019,1/4/2019
3,792,9869,8118,1/1/2019,1/4/2019
4,793,6866,8118,1/1/2019,1/4/2019


In [71]:
df2.head()

,warehouse_id,warehouse_alias,maximum_capacity,employee_total,state
0,1543,Somerset Fulfillment Center,210,14,KY
1,2270,Bowling Green Warehouse,280,13,KY
2,2666,Lansing Fulfillment Center,290,16,MI
3,3417,Gatlinburg Warehouse,620,6,TN
4,3961,Lansing Storage Warehouse,740,22,MI


In [72]:
#inner join
result_inner = pd.merge(df1, df2, on='warehouse_id', how='inner')


In [73]:
#result_join 
result_left=pd.merge(df1, df2, on='warehouse_id', how='left')

In [74]:
result_left

,order_id,customer_id,warehouse_id,order_date,shipper_date,warehouse_alias,maximum_capacity,employee_total,state
0,789,3731,8118,01/01/2019,01/04/2019,Ann Arbor Fulfillment Center,780,17,MI
1,790,3486,8118,1/1/2019,1/4/2019,Ann Arbor Fulfillment Center,780,17,MI
2,791,2623,8118,1/1/2019,1/4/2019,Ann Arbor Fulfillment Center,780,17,MI
3,792,9869,8118,1/1/2019,1/4/2019,Ann Arbor Fulfillment Center,780,17,MI
4,793,6866,8118,1/1/2019,1/4/2019,Ann Arbor Fulfillment Center,780,17,MI
...,...,...,...,...,...,...,...,...,...
9994,10783,8554,9080,12/30/2019,1/2/2020,Frankfort Fulfillment Center,235,5,KY
9995,10784,6439,6509,12/30/2019,1/2/2020,Memphis Fulfillment Center,755,22,TN
9996,10785,4379,6509,12/30/2019,1/2/2020,Memphis Fulfillment Center,755,22,TN
9997,10786,1799,6509,12/30/2019,1/2/2020,Memphis Fulfillment Center,755,22,TN


In [75]:
result_left.duplicated().sum()

0

In [76]:
result_left.isnull().sum()

order_id            0
customer_id         0
warehouse_id        0
order_date          0
shipper_date        0
warehouse_alias     0
maximum_capacity    0
employee_total      0
state               0
dtype: int64

In [77]:
result_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_id          9999 non-null   int64 
 1   customer_id       9999 non-null   int64 
 2   warehouse_id      9999 non-null   int64 
 3   order_date        9999 non-null   object
 4   shipper_date      9999 non-null   object
 5   warehouse_alias   9999 non-null   object
 6   maximum_capacity  9999 non-null   int64 
 7   employee_total    9999 non-null   int64 
 8   state             9999 non-null   object
dtypes: int64(5), object(4)
memory usage: 781.2+ KB


In [104]:
#export
result_left.to_csv('orderswarehouse.csv', index=False)

In [78]:
#find warehouse with the highest order volume 
x=result_left['warehouse_alias'].value_counts().reset_index().rename(columns={'index':'Warehouse Name','warehouse_alias':'Order Volume'})
x

,Warehouse Name,Order Volume
0,Lansing Fulfillment Center,3178
1,Ann Arbor Fulfillment Center,3027
2,Memphis Fulfillment Center,2403
3,Somerset Fulfillment Center,548
4,Frankfort Fulfillment Center,500
5,Knoxville Fulfillment Center,343


In [79]:
result_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_id          9999 non-null   int64 
 1   customer_id       9999 non-null   int64 
 2   warehouse_id      9999 non-null   int64 
 3   order_date        9999 non-null   object
 4   shipper_date      9999 non-null   object
 5   warehouse_alias   9999 non-null   object
 6   maximum_capacity  9999 non-null   int64 
 7   employee_total    9999 non-null   int64 
 8   state             9999 non-null   object
dtypes: int64(5), object(4)
memory usage: 781.2+ KB


In [80]:
#find order process time 
#convert order date and shipper date to dateframe objects
result_left['order_date']=pd.to_datetime(result_left['order_date'])
result_left['shipper_date']=pd.to_datetime(result_left['shipper_date'])
#calculate the process time for each order
result_left['processing_time']=(result_left['order_date']-result_left['shipper_date']).dt.days
#calculate average processing time for each warehouse
average_processing= result_left.groupby('warehouse_alias')['processing_time'].mean().reset_index()
average_processing


,warehouse_alias,processing_time
0,Ann Arbor Fulfillment Center,-3.0
1,Frankfort Fulfillment Center,-3.0
2,Knoxville Fulfillment Center,-3.0
3,Lansing Fulfillment Center,-3.0
4,Memphis Fulfillment Center,-3.0
5,Somerset Fulfillment Center,-3.0


In [190]:
# calculate the total order of each warehouse
x=result_left.groupby('warehouse_alias')['order_id'].count().reset_index().sort_values(by='order_id', ascending=False)
x

,warehouse_alias,order_id
3,Lansing Fulfillment Center,3178
0,Ann Arbor Fulfillment Center,3027
4,Memphis Fulfillment Center,2403
5,Somerset Fulfillment Center,548
1,Frankfort Fulfillment Center,500
2,Knoxville Fulfillment Center,343


In [191]:
#find order per month 
result_left.groupby(result_left['order_date'].dt.to_period("M")).size().reset_index().rename(columns={'order_date':'Month',0:'Total Orders'})

,Month,Total Orders
0,2019-01,859
1,2019-02,805
2,2019-03,850
3,2019-04,806
4,2019-05,800
5,2019-06,786
6,2019-07,844
7,2019-08,871
8,2019-09,842
9,2019-10,906


In [107]:
result_left.groupby('state')['employee_total'].sum().reset_index()

,state,employee_total
0,KY,10172
1,MI,102307
2,TN,57325


In [121]:
result_left['month']=result_left['order_date'].dt.month
result_left.groupby(['month','warehouse_alias'])['order_id'].count().reset_index()

,month,warehouse_alias,order_id
0,1,Ann Arbor Fulfillment Center,267
1,1,Frankfort Fulfillment Center,33
2,1,Knoxville Fulfillment Center,30
3,1,Lansing Fulfillment Center,265
4,1,Memphis Fulfillment Center,219
...,...,...,...
67,12,Frankfort Fulfillment Center,50
68,12,Knoxville Fulfillment Center,29
69,12,Lansing Fulfillment Center,269
70,12,Memphis Fulfillment Center,195


In [201]:
# Extract the month from the 'order_date' column and create a new 'month' column
result_left['month'] = result_left['order_date'].dt.month

x=result_left.groupby(['warehouse_alias','month'])['order_id'].count().reset_index().merge(result_left.groupby('warehouse_alias')['maximum_capacity'].first().reset_index(), on='warehouse_alias', how='left').rename(columns={'order_id':'order_counts'})
# calculate percentage of total order counts 
x['percentage']=(x['order_counts']/x['maximum_capacity'])*100
x

                                    
                                    

,warehouse_alias,month,order_counts,maximum_capacity,percentage
0,Ann Arbor Fulfillment Center,1,267,780,34.230769
1,Ann Arbor Fulfillment Center,2,243,780,31.153846
2,Ann Arbor Fulfillment Center,3,260,780,33.333333
3,Ann Arbor Fulfillment Center,4,271,780,34.743590
4,Ann Arbor Fulfillment Center,5,251,780,32.179487
...,...,...,...,...,...
67,Somerset Fulfillment Center,8,52,210,24.761905
68,Somerset Fulfillment Center,9,41,210,19.523810
69,Somerset Fulfillment Center,10,48,210,22.857143
70,Somerset Fulfillment Center,11,47,210,22.380952


NameError: name 'df' is not defined